In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

import numpy as np
from scipy.io import loadmat

sys.path.insert(0, '..')
from commons.factor import Factor

import helper
import solution
import submit

In [3]:
# Please copy all mat files for these assignments downloaded from Coursea into ./data directory beforehand.

sample = loadmat("./data/PA4Sample.mat", simplify_cells=True)

## Compute Initial Potentials

In [4]:
# This cell contains data prepration code.

inp = sample['InitPotential']['INPUT']
out = sample['InitPotential']['RESULT']

out = {
    'adj_list': helper.adj_matrix_to_adj_list(out['edges']),
    'clique_list': [Factor.from_matlab(f) for f in out['cliqueList']]
}

nodes = [set(x-1) for x in inp['nodes']]

edges = []
edge_mat = inp['edges']
for i in range(len(edge_mat)):
    for j in range(i+1, len(edge_mat)):
        if edge_mat[i, j] == 1:
            edges.append((nodes[i], nodes[j]))

skeleton = {'nodes': nodes, 'edges': edges, 
            'factor_list': [Factor.from_matlab(f) for f in inp['factorList']]}

In [5]:
sol = solution.compute_initial_potentials(skeleton)
out == sol

True

## Get Next Clique

In [6]:
# This cell contains data prepration code.

inp1, inp2 = sample['GetNextC']['INPUT1'], sample['GetNextC']['INPUT2']
out = sample['GetNextC']['RESULT1']-1, sample['GetNextC']['RESULT2']-1

clique_tree = {'adj_list': helper.adj_matrix_to_adj_list(inp1['edges']),
               'clique_list': 'NOT REQUIRED'}


N = len(clique_tree['adj_list'])
msgs = {i: {} for i in range(N)}
for i in range(N):
    for j in range(N):
        msg = inp2[i, j]
        if isinstance(msg.val, int):
            continue
        msg = Factor.from_mat_struct(msg)
        msgs[j][i] = msg

In [7]:
out == solution.get_next_clique(clique_tree, msgs)

True

## Clique Tree Calibrate (Sum Product)

In [8]:
# This cell contains data prepration code.

inp = sample['SumProdCalibrate']['INPUT']
out = sample['SumProdCalibrate']['RESULT']

clique_tree = {'clique_list':  [Factor.from_matlab(f) for f in inp['cliqueList']],
               'adj_list': helper.adj_matrix_to_adj_list(inp['edges'])}
out = {'clique_list':  [Factor.from_matlab(f) for f in out['cliqueList']],
               'adj_list': helper.adj_matrix_to_adj_list(out['edges'])}

In [9]:
solution.clique_tree_calibrate(clique_tree, is_max=0) == out

True

## compute_exact_marginals_bp

In [10]:
# This cell contains data prepration code.

inp = [Factor.from_matlab(f) for f in sample['ExactMarginal']['INPUT']]
out = [Factor.from_matlab(f) for f in sample['ExactMarginal']['RESULT']]

In [11]:
out == solution.compute_exact_marginals_bp(inp, is_max=0)

True

## Factor Max Marginalization

In [12]:
# This cell contains data prepration code.

inp1 = Factor.from_matlab(sample['FactorMax']['INPUT1'])
inp2 = sample['FactorMax']['INPUT2']-1
out = Factor.from_matlab(sample['FactorMax']['RESULT'])

In [13]:
out == solution.factor_max_marginalization(inp1, [inp2, ])

True

## Clique Tree Calibrate (Max Sum)

__NOTE__: Please test your PA4Sample here, as it won't be passed to the grader. Grader has some bug in it that causes error even if your output is correct. 
You will lose 8 marks but such is life.

In [14]:
# This cell contains data prepration code.

inp = sample['MaxSumCalibrate']['INPUT']
out = sample['MaxSumCalibrate']['RESULT']

clique_tree = {'clique_list': [Factor.from_matlab(clique) for clique in inp['cliqueList']], 
               'adj_list': helper.adj_matrix_to_adj_list(inp['edges'])}
out = {'clique_list':  [Factor.from_matlab(f) for f in out['cliqueList']],
               'adj_list': helper.adj_matrix_to_adj_list(out['edges'])}

In [15]:
out == solution.clique_tree_calibrate(clique_tree, is_max=1)

/personal/Courses/Coursera - Probabilistic Graphical Models/PythonAssignments-Unsolved/Exact-Inference/../commons/factor.py:238: RuntimeWarning: divide by zero encountered in log
  new_factor[assignment] = np.log(self.val[assignment])
/personal/Courses/Coursera - Probabilistic Graphical Models/PythonAssignments-Unsolved/Exact-Inference/../commons/factor.py:86: RuntimeWarning: invalid value encountered in double_scalars
  if abs(self[assignment] - other[other_assignment]) > 1e-6:


True

## Compute Exact Max-Marginals

In [16]:
# This cell contains data prepration code.

inp = [Factor.from_matlab(f) for f in sample['MaxMarginals']['INPUT']]
out = [Factor.from_matlab(f) for f in  sample['MaxMarginals']['RESULT']]

In [17]:
out == solution.compute_exact_marginals_bp(inp, is_max=1)

True

## Max Decoding

In [18]:
# This cell contains data prepration code.

inp = [Factor.from_matlab(f) for f in sample['MaxDecoded']['INPUT']]
out = sample['MaxDecoded']['RESULT'] - 1

In [19]:
out == solution.max_decoding(inp)

array([ True,  True,  True,  True,  True,  True,  True])